# Rt2_assignment2

# Code

In [1]:
import rospy
import time
import jupyros as jr
import matplotlib
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np 
import math
import actionlib
import actionlib.msg
import ipywidgets as widgets
from rt2_assignment1.srv import Command, SetVel
from geometry_msgs.msg import Twist
from nav_msgs.msg import Odometry


%matplotlib widget
from matplotlib.animation import FuncAnimation
from matplotlib import animation, rc
from std_msgs.msg import String, Float64, Bool, Float32
from IPython.display import display
from ipywidgets import Button, GridBox, Layout, ButtonStyle, FloatSlider, HBox, VBox
from tf import transformations


## Classes

### ListLim

In [2]:
class ListLim(list):
    def __init__(self, max_size, l=[]):
        super().__init__(l)
        self.max_size = max_size
    def append(self, data):
        super().append(data)
        if super().__len__() > self.max_size:
            super().pop(0)

### Direction Pad

In [3]:
class DirectionPad:
    def __init__(self, lin_slider, ang_slider):
        self.vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=1)
        self.twist = Twist()
        self.go_on_mov = 0.0
        self.ang_mov = 0.0
        self.lin_slider = lin_slider
        self.ang_slider = ang_slider
    
    def goForward(self):
        self.go_on_mov = 1.0
        self.ang_mov = 0.0
    def goBackward(self):
        self.go_on_mov = -1.0
        self.ang_mov = 0.0
    def turnRight(self):
        self.go_on_mov = 0.0
        self.ang_mov = 1.0
    def turnLeft(self):
        self.go_on_mov = 0.0
        self.ang_mov = -1.0
    def stopping(self):
        self.go_on_mov = 0.0
        self.ang_mov = 0.0
    
    def pubTwist(self):
        self.twist.linear.x = self.go_on_mov*lin_slider.value
        self.twist.linear.z = self.ang_mov*ang_slider.value
        
        self.vel_pub.publish(self.twist)

## Global variables

In [4]:
output = widgets.Output()

lin_init = 0.6
ang_init = 0.6

rob_odom_max_leng = 2000
rob_odom_x = ListLim(rob_odom_max_leng)
rob_odom_y = ListLim(rob_odom_max_leng)
rob_odom_rot = 0

vel_max_len = 200
vel_x_sim = ListLim(vel_max_len, [lin_init]*vel_max_len)
vel_x_odom = ListLim(vel_max_len, [0]*vel_max_len)
vel_w_sim = ListLim(vel_max_len, [ang_init]*vel_max_len)
vel_w_odom = ListLim(vel_max_len, [0]*vel_max_len)

t = np.linspace(0,1,vel_max_len)

reached = 0
cancelled = 0
status = [0,0]
time_to_reach = []

from enum import Enum

class Running(Enum):
    STOP = 0
    GO = 1
    PAD = 2
running = Running.STOP

vel_client = None
ui_client = None

## Widgets

### Sliders

In [5]:
max_vel = 1.6
lin_slider = FloatSlider(description='Linear Velocity', value=lin_init, min=0.1, max=max_vel, step=0.02)
ang_slider = FloatSlider(description='Angular Velocity', value=ang_init, min=0.1, max=max_vel, step=0.02)

v1=VBox([lin_slider, ang_slider])
def lin_vel_change(vel):
    vel_client(vel["new"], vel_w_sim[-1])

def ang_vel_change(vel):
    vel_client(vel_x_sim[-1], vel["new"])
    
lin_slider.observe(lin_vel_change, names='value')
ang_slider.observe(ang_vel_change, names='value')

### Buttons

In [6]:
up_Button = Button(description='\u2191', 
                  layout=Layout(width='auto', grid_area='up'),
                  style=ButtonStyle(button_color='lightblue'))
down_Button = Button(description='\u2193',
                  layout=Layout(width='auto', grid_area='down'),
                  style=ButtonStyle(button_color='lightblue'))
right_Button = Button(description='\u2192', 
                  layout=Layout(width='auto', grid_area='right'),
                  style=ButtonStyle(button_color='lightblue'))
left_Button = Button(description='\u2190', 
                  layout=Layout(width='auto', grid_area='left'),
                  style=ButtonStyle(button_color='lightblue'))


gb = GridBox(children=[up_Button, down_Button, right_Button, left_Button],
        layout=Layout(
            width='100%',
            grid_template_rows='auto auto',
            grid_template_columns='33% 33% 33%',
            grid_template_areas='''
            ". up ."
            "left down right"
            ''')
        )

In [7]:
start_Button = Button(description='START', layout=Layout(width='100%',grid_area='start'),
                  style=ButtonStyle(button_color='lightblue'))
stop_Button = Button(description='STOP', layout=Layout(width='100%',grid_area='stop'),
                  style=ButtonStyle(button_color='lightblue'))
h1=HBox([start_Button,stop_Button])

g_all = GridBox(children=[v1, h1, gb],
            layout=Layout(
                width='100%',
                grid_template_row='auto auto',
                grid_template_columns='33% 40% 27%',
                grid_templates_areas='''
                "v1 . gb"
                " . h1 ."
                ''')
            )

## Functions

In [8]:
dir_pad = DirectionPad(lin_slider, ang_slider)

def start_gotopoint(b):
    global running
    if running != Running.GO:
        ui_client("start")
        running = Running.GO

def stop_motion(b):
    global running
    if running == Running.GO:
        ui_client("stop")
    elif running == Running.PAD:
        dir_pad.stop()
        dir_pad.pubTwist()
        
    running = Running.STOP

def  directionalButton(b):
    global running
    if running == Running.GO:
        stop_motion(b)
    if b.layout.grid_area == 'up':
        dir_pad.goForward()
    elif b.layout.grid_area == 'down':
        dir_pad.goBackward()
    elif b.layout.grid_area == 'right':
        dir_pad.turnRight()
    elif b.layout.grid_area == 'left':
        dir_pad.turnLeft()
    else:
        return 

    running = Running.PAD

def odomFunction(msg):
    global rob_odom_rot, rob_odom_x, rob_odom_y
    
    global running
    rob_odom_x.append(msg.pose.pose.position.x)
    rob_odom_y.append(msg.pose.pose.position.y)
    
    quaternion = (
        msg.pose.pose.orientation.x,
        msg.pose.pose.orientation.y,
        msg.pose.pose.orientation.z,
        msg.pose.pose.orientation.w)
    euler = transformations.euler_from_quaternion(quaternion)
    rob_odom_rot = euler[2]
    
    vel_x_sim.append(lin_slider.value)
    vel_x_odom.append(msg.twist.twist.linear.x)
    
    vel_w_sim.append(ang_slider.value)
    vel_w_odom.append(msg.twist.twist.angular.x)
    
    if running == Running.PAD:
        dir_pad.pubTwist()

def reach_function(msg):
    global reached, cancelled, status
    if msg.data:
        reached+=1
    else:
        cancelled+=1
    status=[reached, cancelled]
    
def ttr_function(msg):
    global time_to_reach
    time_to_reach.append(msg.data)

### Buttons binding

In [9]:
start_Button.on_click(start_gotopoint)
stop_Button.on_click(stop_motion)
up_Button.on_click(directionalButton)
down_Button.on_click(directionalButton)
left_Button.on_click(directionalButton)
right_Button.on_click(directionalButton)

## Ros node start

In [10]:
rospy.init_node("jupyter_interface")

sub = rospy.Subscriber('/odom', Odometry, odomFunction)

rospy.wait_for_service('/user_interface')
ui_client = rospy.ServiceProxy('/user_interface', Command)
rospy.wait_for_service('/set_vel')
vel_client = rospy.ServiceProxy('/set_vel', SetVel)

## Control panel

In [21]:
display(g_all)
output

GridBox(children=(VBox(children=(FloatSlider(value=0.6, description='Linear Velocity', max=1.6, min=0.1, step=…

Output()

# Plots

## Position and orientation

In [27]:
fig, ax = plt.subplots()

ax.set_xlim((-5.2, 5.2))
ax.set_ylim((-5.2, 5.2))

odom_plt, = ax.plot([], [], 'r-')
arrw_plt, = ax.plot([], [], 'g-.', linewidth=2.5)
arrw_len_ = 0.7

ax.set_title("Robot odometry pose")
ax.set_xlabel("X")
ax.set_ylabel("Y")

def init():
    odom_plt.set_data([], [])
    arrw_plt.set_data([], [])
    return (odom_plt, arrw_plt)

def animate(i):
    odom_plt.set_data(rob_odom_x, rob_odom_y)
    n_x = rob_odom_x[-1] + math.cos(rob_odom_rot)*arrw_len_
    n_y = rob_odom_y[-1] + math.sin(rob_odom_rot)*arrw_len_
    
    arrw_plt.set_data((rob_odom_x[-1], n_x), (rob_odom_y[-1], n_y))
    
    return (odom_plt, arrw_plt)

anim = animation.FuncAnimation(fig, animate, init_func = init, frames=50, interval=20, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Velocities

In [23]:
fig_vel, axs = plt.subplots(nrows=2)

ax0, ax1 = axs.flatten()
ax0.set_title("Linear velocity")
ax0.set_ylim(0.0, max_vel)
ax0.set_xticks([])

vel_x_plt_sim, = ax0.plot([], [], 'r', label='max linear velocity')
vel_x_plt_odm, = ax0.plot([], [], 'g', label='current linear velocity', linewidth=2.0)

ax1.set_title("Angular velocity")
ax1.set_ylim(0.0, max_vel)
ax1.set_xticks([])

vel_w_plt_sim, = ax1.plot([], [], 'r', label='max angular velocity')
vel_w_plt_odm, = ax1.plot([], [], 'g', label='current angular velocity', linewidth=2.0)


def init_vel():
    vel_x_plt_sim.set_data([], [])
    vel_x_plt_odm.set_data([], [])
    
    vel_w_plt_sim.set_data([], [])
    vel_w_plt_odm.set_data([], [])
    return (vel_x_plt_sim, vel_x_plt_odm, vel_w_plt_sim, vel_w_plt_odm)

def animate_vel(i):
    vel_x_plt_sim.set_data(t, vel_x_sim)
    vel_x_plt_odm.set_data(t, vel_x_odom)
    
    vel_w_plt_sim.set_data(t, vel_w_sim)
    vel_w_plt_odm.set_data(t, vel_w_odom)
    
    return (vel_x_plt_sim, vel_x_plt_odm, vel_w_plt_sim, vel_w_plt_odm)

anim_vel = animation.FuncAnimation(fig_vel, animate_vel, init_func = init_vel, frames=100, interval=20, blit=True)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Reached vs Cancelled

In [25]:


# setting the figure properties
fig3,ax3 = plt.subplots()
legend = ['reached', 'cancelled']
ax3.bar(legend,status,color = 'r', width = 0.5)
ax3.set_title("Reached and Cancelled goal")

# function called to initialize the plot
def init3():
    ax3.bar(legend,stats,color = 'r', width = 0.5)

# function called to update periodically the plot
def animate3(i):
    global stats, ax3
    ax3.bar(legend,stats,color = 'r', width = 0.5)
    
ani3=animation.FuncAnimation(fig3, animate3, init_func=init3, frames=100, interval=100, blit=True)




Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Time to reach

In [26]:


# setting the figure properties
fig4,ax4= plt.subplots()
ax4.hist(time_to_reach, 10, label='time to reach goal', color='red',lw=10)
ax4.legend()
ax4.set_title("Time to reach Goal")
ax4.set_xlabel('time (s)')

# function called to initialize the plot
def init4():
    ax4.hist(time_to_reach, 10, label='time to reach goal', color='red',lw=10)

# function called to update periodically the plot    
def animate4(i):
    global time_to_reach, ax4
    ax4.hist(time_to_reach,10, color='red',lw=10)
    
ani4=animation.FuncAnimation(fig4, animate4,init_func=init4, frames=100, interval=100, blit=True)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …